In [37]:
import ast
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from sklearn.model_selection import KFold
import joblib

In [38]:
# abrindo os dados de treinamento
df = pd.read_csv("./../ansatz_result/data.csv")
X = df.drop(columns=["target"]).to_numpy()
y = pd.DataFrame(df['target'].apply(ast.literal_eval).tolist()).to_numpy()

# y será uma lista de probabilidades com o melhor ansatz
# essa probabilidade de cada ansatz i sera definida como prob(i) = ansatz(i)/soma de todos os ansatz
y_prob = y / y.sum(axis=1, keepdims=True)

In [39]:
# criando kfold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

#realizando validação cruzada

dt_regressor_classifier = DecisionTreeRegressor(random_state=45)

score = cross_val_score(dt_regressor_classifier, X, y, cv=kf)

print("Scores:", score)
print("Média:", score.mean())
print("Desvio padrão:", score.std())

Scores: [0.77098979 0.72907267 0.77288125]
Média: 0.7576479027426459
Desvio padrão: 0.020220491574825535


In [40]:
# treinando o dt classifier e obtendo os top3 ansatz mais classificados
dt_regressor_classifier.fit(X, y_prob)


# 3. Prever e rankear
def predict_rank_ansatz(model, X_new, k=3):
    probas = model.predict(X_new.reshape(1, -1))[0]
    topk = np.argsort(probas)[-k:][::-1]
    return topk, probas[topk]

# Rankear os top-3 ansatz com maiores probabilidades
top3_idx, top3_probs = predict_rank_ansatz(dt_regressor_classifier, X[6])

print(f"Top-3 Ansatz: {top3_idx} com probabilidades {top3_probs}")

Top-3 Ansatz: [20  0  4] com probabilidades [0.03810976 0.03810976 0.03810976]


In [ ]:
# salvar o modelo treinado
joblib.dump(dt_regressor_classifier, './models_salvos/dt_regressor_classifier.joblib')

['./dt_regressor_classifier.joblib']